1. load the dataset

In [1]:
import boto3
import pandas as pd

bucket = "ai-bmi-predictor-v2"
key = "test-data/eff_testingA_v3.csv"

s3 = boto3.client("s3")
obj = s3.get_object(Bucket=bucket, Key=key)

data = pd.read_csv(obj["Body"])

data.head()


,photo_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,shoulder-breadth,shoulder-to-crotch,thigh,waist,wrist,gender,height_cm,weight_kg,front_silhouette_area_norm,side_silhouette_area_norm
0,e5ae8fe5bbdf611a1e8d06e66e849bdf,0.073159,0.085775,-0.133776,0.881202,0.214236,0.016104,-0.180302,-0.100713,-0.117249,...,39.922305,70.005128,55.945992,98.250390,20.187082,male,180.00,94.60,0.194761,0.117846
1,605a5fd09058c48156b0ef518b63b2de,0.092031,-0.066016,-0.145132,0.687441,0.186508,-0.075221,-0.093846,-0.035840,0.033903,...,39.974203,73.591637,55.397032,88.003618,17.715785,male,188.90,86.75,0.153597,0.138233
2,909c9277309e13ee014e347603aba620,0.057046,-0.051366,-0.148253,0.675916,0.209973,-0.073485,-0.072783,-0.059395,0.008370,...,36.059983,66.440526,53.742692,82.100598,17.086464,male,179.70,73.85,0.170371,0.105665
3,bef6a68bc8dd475c124f6de2413385d3,-0.018792,0.016435,-0.148091,0.464433,0.242849,-0.106556,0.001489,-0.083478,0.096048,...,34.071748,62.218026,52.396573,83.999124,16.299751,female,166.95,69.05,0.174890,0.117724
4,6d7ed4bc4a17546447efed0ca6e2ff11,0.084419,0.065945,-0.153379,0.635377,0.285274,-0.056372,-0.139008,-0.120711,-0.002466,...,36.834735,64.426273,49.895157,86.020117,16.531431,male,173.20,65.55,0.171429,0.109745


2. data preprocessing

2.1. categorical encoding for 'gender' feature

In [2]:
import pandas as pd                 # import pandas for data handling

data['gender'] = data['gender'].astype('category')  # convert 'gender' values to categorical type
data['gender'] = data['gender'].cat.codes           # replace 'gender' with its numeric category codes

In [3]:
data['gender'].head()

0    1
1    1
2    1
3    0
4    1
Name: gender, dtype: int8

In [4]:
#data['height_cm'].head()

2.2. define weight frequencies for class imbalance issue for weight_kg

In [5]:
'''import pandas as pd                     # import pandas for data handling
import numpy as np                      # import numpy to help with safe division

# Assume 'data' is your DataFrame and already loaded
#print("Preview of data:\n", data.head())  # print first few rows to check data
print("\nTotal samples in dataset:", len(data))  # print total number of rows

# -----------------------------
# 1. Create boolean masks for the three weight_kg classes
# -----------------------------
class_1_mask = data['weight_kg'] < 60                      # True where weight_kg is less than 60
class_2_mask = data['weight_kg'] > 100                     # True where weight_kg is greater than 100
class_3_mask = (data['weight_kg'] >= 60) & (data['weight_kg'] <= 100)  # True where weight is between 60 and 100

# -----------------------------
# 2. Calculate class frequencies (counts)
# -----------------------------
freq_class_1 = class_1_mask.sum()          # number of samples with weight_kg < 60
freq_class_2 = class_2_mask.sum()          # number of samples with weight_kg > 100
freq_class_3 = class_3_mask.sum()          # number of samples with 60 <= weight_kg <= 100

print("\nClass frequencies:")              # header for clarity
print("Class 1 (weight_kg < 60):", freq_class_1)   # print frequency of class 1
print("Class 2 (weight_kg > 100):", freq_class_2)  # print frequency of class 2
print("Class 3 (60 <= weight_kg <= 100):", freq_class_3)  # print frequency of class 3

# -----------------------------
# 3. Number of classes according to the strategy
# -----------------------------
num_classes = 3                             # we defined three classes by the rules above
print("\nNumber of classes:", num_classes)  # print number of classes

# -----------------------------
# 4. Compute inverse-frequency weights for each class
#    Formula: w = total_samples / (num_classes * class_frequency)
# -----------------------------
total_samples = len(data)                   # total number of rows in the dataset

def safe_weight(class_freq):                # helper function to avoid division by zero
    if class_freq == 0:                     # check if a class has zero samples
        return np.nan                       # return NaN if no samples exist for that class
    return total_samples / (num_classes * class_freq)  # apply weighting formula

weight_class_1 = safe_weight(freq_class_1)  # compute weight for class 1
weight_class_2 = safe_weight(freq_class_2)  # compute weight for class 2
weight_class_3 = safe_weight(freq_class_3)  # compute weight for class 3

print("\nClass weights (inverse frequency):")          # header for class weights
print("Weight for Class 1 (weight_kg < 60):", weight_class_1)   # print weight of class 1
print("Weight for Class 2 (weight_kg > 100):", weight_class_2)  # print weight of class 2
print("Weight for Class 3 (60 <= weight_kg <= 100):", weight_class_3)  # print weight of class 3
'''

'import pandas as pd                     # import pandas for data handling\nimport numpy as np                      # import numpy to help with safe division\n\n# Assume \'data\' is your DataFrame and already loaded\n#print("Preview of data:\n", data.head())  # print first few rows to check data\nprint("\nTotal samples in dataset:", len(data))  # print total number of rows\n\n# -----------------------------\n# 1. Create boolean masks for the three weight_kg classes\n# -----------------------------\nclass_1_mask = data[\'weight_kg\'] < 60                      # True where weight_kg is less than 60\nclass_2_mask = data[\'weight_kg\'] > 100                     # True where weight_kg is greater than 100\nclass_3_mask = (data[\'weight_kg\'] >= 60) & (data[\'weight_kg\'] <= 100)  # True where weight is between 60 and 100\n\n# -----------------------------\n# 2. Calculate class frequencies (counts)\n# -----------------------------\nfreq_class_1 = class_1_mask.sum()          # number of sample

2.3. define weight frequencies for class imbalance issue for gender feature

In [6]:
'''import numpy as np                                      # import numpy for numeric utilities (like NaN)

print("Preview of gender column:\n", data['gender'].head())  # show first few gender values to inspect

# -----------------------------------
# 1. Calculate class frequencies for gender
# -----------------------------------
gender_counts = data['gender'].value_counts()           # count how many samples belong to each gender class

print("\nClass frequencies for gender:")                # header for class frequency output
for gender_class, freq in gender_counts.items():        # loop over each gender class and its frequency
    print(f"Class {gender_class}: {freq}")              # print the class label and its frequency

# -----------------------------------
# 2. Number of gender classes
# -----------------------------------
num_gender_classes = len(gender_counts)                 # compute how many distinct gender classes we have
print("\nNumber of gender classes:", num_gender_classes)  # print number of gender classes

# -----------------------------------
# 3. Compute inverse-frequency weights for each gender class
#    Formula: w = total_samples / (num_classes * class_frequency)
# -----------------------------------
total_samples = len(data)                               # total number of samples in the dataset

def safe_weight(class_freq):                            # define helper function to compute class weight safely
    if class_freq == 0:                                 # check for zero frequency to avoid division by zero
        return np.nan                                   # return NaN if a class somehow has zero samples
    return total_samples / (num_gender_classes * class_freq)  # apply the inverse-frequency weight formula

gender_weights = {}                                     # create an empty dictionary to store weights per class
for gender_class, freq in gender_counts.items():        # loop through each gender class and its frequency
    gender_weights[gender_class] = safe_weight(freq)    # compute and store the weight for this gender class

print("\nClass weights (inverse frequency) for gender:")  # header for weight output
for gender_class, weight in gender_weights.items():     # loop over each class and its weight
    print(f"Weight for class {gender_class}: {weight}") # print the computed weight for this gender class
'''

'import numpy as np                                      # import numpy for numeric utilities (like NaN)\n\nprint("Preview of gender column:\n", data[\'gender\'].head())  # show first few gender values to inspect\n\n# -----------------------------------\n# 1. Calculate class frequencies for gender\n# -----------------------------------\ngender_counts = data[\'gender\'].value_counts()           # count how many samples belong to each gender class\n\nprint("\nClass frequencies for gender:")                # header for class frequency output\nfor gender_class, freq in gender_counts.items():        # loop over each gender class and its frequency\n    print(f"Class {gender_class}: {freq}")              # print the class label and its frequency\n\n# -----------------------------------\n# 2. Number of gender classes\n# -----------------------------------\nnum_gender_classes = len(gender_counts)                 # compute how many distinct gender classes we have\nprint("\nNumber of gender class

2.4. weight frequencies for weight classes and gender classes

In [7]:
'''import numpy as np   # import numpy for numeric operations

# -------------------------------------------------
# 1. Store the already-computed weights for weight classes
#    (use the variables you created when handling weight_kg)
# -------------------------------------------------
weight_class_weights = {                          # dictionary to hold weight-class weights
    'weight_<60':  weight_class_1,                # weight for class: weight_kg < 60
    'weight_>100': weight_class_2,                # weight for class: weight_kg > 100
    'weight_60_100': weight_class_3               # weight for class: 60 <= weight_kg <= 100
}

print("Weight-class weights:", weight_class_weights)  # print weight-class weights to check

# gender_weights dict is assumed from previous step, e.g. {0: w0, 1: w1}
print("Gender-class weights:", gender_weights)        # print gender-class weights to check

# -------------------------------------------------
# 2. Multiply each gender class with each weight class
#    wi = w_weight * w_gender
# -------------------------------------------------
combined_weights = {}                                # dictionary to store combined class weights

print("\nCombined weights for each (weight_class, gender_class):")  # header
for w_label, w_w in weight_class_weights.items():    # loop over weight classes
    for g_label, w_g in gender_weights.items():      # loop over gender classes
        wi = w_w * w_g                               # multiply weight and gender class weights
        combined_weights[(w_label, g_label)] = wi    # store in dictionary
        print(f"{w_label} & gender {g_label}: {wi}") # print each combination'''

'import numpy as np   # import numpy for numeric operations\n\n# -------------------------------------------------\n# 1. Store the already-computed weights for weight classes\n#    (use the variables you created when handling weight_kg)\n# -------------------------------------------------\nweight_class_weights = {                          # dictionary to hold weight-class weights\n    \'weight_<60\':  weight_class_1,                # weight for class: weight_kg < 60\n    \'weight_>100\': weight_class_2,                # weight for class: weight_kg > 100\n    \'weight_60_100\': weight_class_3               # weight for class: 60 <= weight_kg <= 100\n}\n\nprint("Weight-class weights:", weight_class_weights)  # print weight-class weights to check\n\n# gender_weights dict is assumed from previous step, e.g. {0: w0, 1: w1}\nprint("Gender-class weights:", gender_weights)        # print gender-class weights to check\n\n# -------------------------------------------------\n# 2. Multiply each ge

2.5. create a dictionary for weights and row index

In [8]:
'''# Check current columns in the DataFrame
print("Columns before adding index column:\n", data.columns)

# Add a new column named 'index' with values from 0 to number_of_rows-1
data['index'] = range(len(data))

# Move 'index' to the front (optional, just for nicer viewing)
cols = ['index'] + [c for c in data.columns if c != 'index']  # build new column order
data = data[cols]                                            # reorder columns

# Show first few rows to verify the new indexing column
#print("\nDataFrame after adding 'index' column:\n", data.head())
'''

'# Check current columns in the DataFrame\nprint("Columns before adding index column:\n", data.columns)\n\n# Add a new column named \'index\' with values from 0 to number_of_rows-1\ndata[\'index\'] = range(len(data))\n\n# Move \'index\' to the front (optional, just for nicer viewing)\ncols = [\'index\'] + [c for c in data.columns if c != \'index\']  # build new column order\ndata = data[cols]                                            # reorder columns\n\n# Show first few rows to verify the new indexing column\n#print("\nDataFrame after adding \'index\' column:\n", data.head())\n'

In [9]:
'''import numpy as np               # import numpy for numeric operations
import pickle                    # import pickle to save Python objects

# -------------------------------------------------
# 0. We assume these already exist:
#    - weight_class_1, weight_class_2, weight_class_3
#    - gender_weights   (dict: {gender_class: weight})
# -------------------------------------------------

# create a dictionary of weight-class weights (same as before)
weight_class_weights = {         # dictionary mapping weight class labels to their weights
    'weight_<60':  weight_class_1,      # weight for class: weight_kg < 60
    'weight_>100': weight_class_2,      # weight for class: weight_kg > 100
    'weight_60_100': weight_class_3     # weight for class: 60 <= weight_kg <= 100
}

print("Weight-class weights:", weight_class_weights)  # print weight-class weights
print("Gender-class weights:", gender_weights)        # print gender-class weights

# -------------------------------------------------
# 1. Helper function to get the weight class label for a given weight_kg
# -------------------------------------------------
def get_weight_class(w):         # define a function that receives a single weight value
    if w < 60:                   # check if weight is less than 60
        return 'weight_<60'      # return label for class 1
    elif w > 100:                # check if weight is greater than 100
        return 'weight_>100'     # return label for class 2
    else:                        # otherwise weight is between 60 and 100 (inclusive)
        return 'weight_60_100'   # return label for class 3

# -------------------------------------------------
# 2. Build dictionary: keys = index values, values = combined weights
# -------------------------------------------------
final_weights = {}               # create empty dictionary to store final weights

print("\nBuilding final_weights dictionary...")  # message to track progress

for _, row in data.iterrows():   # loop over each row of the DataFrame
    idx_val = row['index']       # get the value from the 'index' column for this row
    gender_val = row['gender']   # get the gender class value for this row
    weight_val = row['weight_kg']# get the weight_kg value for this row

    w_class = get_weight_class(weight_val)        # determine weight class label from weight_kg
    w_weight = weight_class_weights[w_class]      # look up the weight-class weight
    w_gender = gender_weights[gender_val]         # look up the gender-class weight

    combined_w = w_weight * w_gender             # multiply to get combined weight w_i
    final_weights[idx_val] = combined_w          # store combined weight in dictionary with key=index

print("Number of entries in final_weights:", len(final_weights))  # print number of entries
print("First 5 items in final_weights:", list(final_weights.items())[:5])  # show first few items

# -------------------------------------------------
# 3. Check index 0: gender, weight_kg, and combined weight
# -------------------------------------------------
print("\nChecking entry with index 0...")        # message to show what we're doing

row0 = data.loc[data['index'] == 0].iloc[0]      # select the row where 'index' column equals 0

gender0 = row0['gender']                         # get gender value for index 0
weight0 = row0['weight_kg']                      # get weight_kg value for index 0
w_class0 = get_weight_class(weight0)             # get weight class label for index 0

w_weight0 = weight_class_weights[w_class0]       # get weight-class weight for index 0
w_gender0 = gender_weights[gender0]              # get gender-class weight for index 0
combined0_calc = w_weight0 * w_gender0           # calculate combined weight for index 0

print("Row 0 -> gender:", gender0)               # print gender class for index 0
print("Row 0 -> weight_kg:", weight0)            # print weight_kg for index 0
print("Row 0 -> weight class:", w_class0)        # print weight class label for index 0
print("w_weight for row 0:", w_weight0)          # print weight-class weight for index 0
print("w_gender for row 0:", w_gender0)          # print gender-class weight for index 0
print("Combined weight (calculated):", combined0_calc)        # print calculated combined weight
print("Combined weight from final_weights[0]:", final_weights[0])  # print value from dictionary

# -------------------------------------------------
# 4. Save final_weights dictionary as a pickle file
# -------------------------------------------------
print("\nSaving final_weights dictionary as pickle file...")   # message to track saving step

with open('final_weights.pkl', 'wb') as f:       # open a file named 'final_weights.pkl' in binary write mode
    pickle.dump(final_weights, f)                # write dictionary to the file using pickle

print("Dictionary saved to 'final_weights.pkl'.")# confirmation message
'''

'import numpy as np               # import numpy for numeric operations\nimport pickle                    # import pickle to save Python objects\n\n# -------------------------------------------------\n# 0. We assume these already exist:\n#    - weight_class_1, weight_class_2, weight_class_3\n#    - gender_weights   (dict: {gender_class: weight})\n# -------------------------------------------------\n\n# create a dictionary of weight-class weights (same as before)\nweight_class_weights = {         # dictionary mapping weight class labels to their weights\n    \'weight_<60\':  weight_class_1,      # weight for class: weight_kg < 60\n    \'weight_>100\': weight_class_2,      # weight for class: weight_kg > 100\n    \'weight_60_100\': weight_class_3     # weight for class: 60 <= weight_kg <= 100\n}\n\nprint("Weight-class weights:", weight_class_weights)  # print weight-class weights\nprint("Gender-class weights:", gender_weights)        # print gender-class weights\n\n# --------------------

2.6. apply stnadard sclaer for body measurements and robust scaler for cnn extracted features

In [10]:
data.head()

,photo_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,shoulder-breadth,shoulder-to-crotch,thigh,waist,wrist,gender,height_cm,weight_kg,front_silhouette_area_norm,side_silhouette_area_norm
0,e5ae8fe5bbdf611a1e8d06e66e849bdf,0.073159,0.085775,-0.133776,0.881202,0.214236,0.016104,-0.180302,-0.100713,-0.117249,...,39.922305,70.005128,55.945992,98.250390,20.187082,1,180.00,94.60,0.194761,0.117846
1,605a5fd09058c48156b0ef518b63b2de,0.092031,-0.066016,-0.145132,0.687441,0.186508,-0.075221,-0.093846,-0.035840,0.033903,...,39.974203,73.591637,55.397032,88.003618,17.715785,1,188.90,86.75,0.153597,0.138233
2,909c9277309e13ee014e347603aba620,0.057046,-0.051366,-0.148253,0.675916,0.209973,-0.073485,-0.072783,-0.059395,0.008370,...,36.059983,66.440526,53.742692,82.100598,17.086464,1,179.70,73.85,0.170371,0.105665
3,bef6a68bc8dd475c124f6de2413385d3,-0.018792,0.016435,-0.148091,0.464433,0.242849,-0.106556,0.001489,-0.083478,0.096048,...,34.071748,62.218026,52.396573,83.999124,16.299751,0,166.95,69.05,0.174890,0.117724
4,6d7ed4bc4a17546447efed0ca6e2ff11,0.084419,0.065945,-0.153379,0.635377,0.285274,-0.056372,-0.139008,-0.120711,-0.002466,...,36.834735,64.426273,49.895157,86.020117,16.531431,1,173.20,65.55,0.171429,0.109745


In [11]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# -----------------------------
# 1. Columns
# -----------------------------

# columns to exclude from any scaling
exclude_cols = ['photo_id', 'subject_id', 'index', 'gender']

# target columns (predicted outputs)
target_cols = [
    'ankle', 'arm-length', 'bicep', 'calf', 'chest', 'forearm', 'hip',
    'leg-length', 'shoulder-breadth', 'shoulder-to-crotch', 'thigh',
    'waist', 'wrist', 'weight_kg'
]

# feature columns that must use StandardScaler (but are NOT targets)
standard_feature_cols = ['height_cm','front_silhouette_area_norm','side_silhouette_area_norm']

# keep only existing columns
target_cols = [c for c in target_cols if c in data.columns]
standard_feature_cols = [c for c in standard_feature_cols if c in data.columns]

# remaining feature columns → RobustScaler
robust_feature_cols = [
    col for col in data.columns
    if col not in exclude_cols
    and col not in target_cols
    and col not in standard_feature_cols
]

# -----------------------------
# 2. Create scalers
# -----------------------------

scaler_targets = StandardScaler()           # for inverse scaling
scaler_standard_features = StandardScaler()
scaler_robust_features = RobustScaler()

# -----------------------------
# 3. Fit & transform
# -----------------------------

data[target_cols] = scaler_targets.fit_transform(data[target_cols])
data[standard_feature_cols] = scaler_standard_features.fit_transform(
    data[standard_feature_cols]
)
data[robust_feature_cols] = scaler_robust_features.fit_transform(
    data[robust_feature_cols]
)


3. model validation

3.1. split the data for independent and dependent features

In [12]:
# List of columns to be used as dependent (target) features
target_cols = [
    'ankle', 'arm-length', 'bicep', 'calf', 'chest', 'forearm', 'hip',
    'leg-length', 'shoulder-breadth', 'shoulder-to-crotch', 'thigh',
    'waist', 'wrist', 'weight_kg'
]

# Select these columns from the DataFrame as the multi-target Y
Y = data[target_cols]                  # Y will hold all dependent variables for multi-target regression

print("Selected target columns:", target_cols)  # print which columns are used as targets
print("Shape of Y (samples, targets):", Y.shape)  # print shape to confirm dimensions

Selected target columns: ['ankle', 'arm-length', 'bicep', 'calf', 'chest', 'forearm', 'hip', 'leg-length', 'shoulder-breadth', 'shoulder-to-crotch', 'thigh', 'waist', 'wrist', 'weight_kg']
Shape of Y (samples, targets): (1684, 14)


In [13]:
# Columns to drop for building independent features (X)
drop_cols = ['photo_id', 'subject_id'] + target_cols   # combine ID columns with target columns

print("Columns to drop for X:\n", drop_cols)           # show which columns will be removed

# Create X by dropping ID columns and all target columns
X = data.drop(columns=drop_cols)                       # drop the unwanted columns to get independent features

print("\nShape of X (samples, independent features):", X.shape)  # print shape of X
#print("\nColumns in X:\n", X.columns.tolist())         # list all feature names in X

Columns to drop for X:
 ['photo_id', 'subject_id', 'ankle', 'arm-length', 'bicep', 'calf', 'chest', 'forearm', 'hip', 'leg-length', 'shoulder-breadth', 'shoulder-to-crotch', 'thigh', 'waist', 'wrist', 'weight_kg']

Shape of X (samples, independent features): (1684, 5124)


In [14]:
X.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,s2555,s2556,s2557,s2558,s2559,s2560,gender,height_cm,front_silhouette_area_norm,side_silhouette_area_norm
0,0.774462,1.040661,0.342109,1.366849,-0.398465,1.118232,-2.134066,-0.676307,-1.995643,0.548878,...,-0.576038,-0.817925,0.264697,-0.373763,-0.786373,0.623649,1,1.066025,1.454634,0.456414
1,1.162176,-0.619511,-0.222455,0.380021,-0.696737,-0.080061,-0.764886,0.635084,0.058381,-0.636982,...,0.647752,0.725138,0.020168,-0.598226,3.017102,-0.008240,1,2.083311,-1.626916,2.355284
2,0.443413,-0.459284,-0.377604,0.321320,-0.444318,-0.057285,-0.431309,0.158932,-0.288579,-1.554653,...,-0.894785,-0.668226,0.509909,-0.659391,0.624502,1.260808,1,1.031734,-0.371225,-0.678056
3,-1.114722,0.282268,-0.369572,-0.755762,-0.090680,-0.491214,0.744916,-0.327898,0.902881,0.414726,...,0.211070,-0.479480,-0.684825,0.051598,-0.377326,0.769187,0,-0.425615,-0.032910,0.445111
4,1.005786,0.823773,-0.632466,0.114854,0.365689,0.167260,-1.480106,-1.080566,-0.435842,-0.041774,...,0.570939,0.856161,3.154351,0.484065,1.333055,-0.715199,1,0.288772,-0.292026,-0.298089


3.2. load the model for inferencing

In [15]:
import boto3                                             # import boto3 to read from S3
import io                                                # import io for in-memory byte streams
import h5py                                              # import h5py to open HDF5 file objects
import tensorflow as tf                                  # import tensorflow for loading the model

bucket_name = "ai-bmi-predictor-v2"                         # S3 bucket name
model_key  = "trained-models/efficientnet-models/eff_ann_version14.h5"  # path of model file in S3

print("Creating S3 client...")                           # status message
s3 = boto3.client("s3")                                  # create S3 client (uses your AWS credentials)

print(f"Reading model bytes from s3://{bucket_name}/{model_key}")  # status message
obj = s3.get_object(Bucket=bucket_name, Key=model_key)   # download object bytes into memory
model_bytes = obj["Body"].read()                         # read the body content as raw bytes

byte_stream = io.BytesIO(model_bytes)                    # wrap bytes in an in-memory binary stream

print("Opening HDF5 file from memory...")                # status message
with h5py.File(byte_stream, 'r') as h5file:              # open the stream as an HDF5 file
    best_model = tf.keras.models.load_model(h5file)      # load Keras model from this HDF5 file object

print("Model loaded successfully from S3 (in-memory)!")  # confirmation message

2025-12-29 10:23:22.104693: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-29 10:23:23.717831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-29 10:23:23.955368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-29 10:23:23.958803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-29 10:23:24.414435: I tensorflow/core/platform/cpu_feature_gua

Creating S3 client...
Reading model bytes from s3://ai-bmi-predictor-v2/trained-models/efficientnet-models/eff_ann_version14.h5
Opening HDF5 file from memory...
Model loaded successfully from S3 (in-memory)!


2025-12-29 10:23:37.083918: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-29 10:23:44.065979: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-29 10:23:44.069256: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [16]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               1312000   
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 14)                3598      
                                                                 
Total params: 1315598 (5.02 MB)
Trainable params: 1315598 (5.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


3.3. calculate performance matrices

In [17]:
# -------------------------------------------
# 1. Imports
# -------------------------------------------
import numpy as np                               # numerical operations
import pandas as pd                              # to build a nice results table
import tensorflow as tf                          # to load and run the Keras model
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error  # metrics

# -------------------------------------------
# 3. Prepare validation data
#    We assume:
#      - X : pandas DataFrame with validation independent features (already scaled)
#      - Y : pandas DataFrame with validation targets (same order as training, already scaled)
# -------------------------------------------

print("Shape of validation X:", X.shape)            # show shape of validation features
print("Shape of validation Y:", Y.shape)            # show shape of validation targets

# Save target column names (body measurement names)
target_cols = list(Y.columns)                       # list of body measurement names
print("\nTarget columns (body measurements):")
print(target_cols)                                  # print body measurement names

# Convert X and Y to NumPy arrays for prediction / metric computation
X_val = X.values.astype("float32")                  # features as float32 array
Y_val = Y.values.astype("float32")                  # targets as float32 array

Shape of validation X: (1684, 5124)
Shape of validation Y: (1684, 14)

Target columns (body measurements):
['ankle', 'arm-length', 'bicep', 'calf', 'chest', 'forearm', 'hip', 'leg-length', 'shoulder-breadth', 'shoulder-to-crotch', 'thigh', 'waist', 'wrist', 'weight_kg']


In [18]:

# -------------------------------------------
# 4. Run inference to get predictions
# -------------------------------------------
print("\nRunning inference on validation data ...") # status message
Y_pred = best_model.predict(X_val, verbose=0)       # model predictions for validation data
print("Inference completed.")
print("Predictions shape:", Y_pred.shape)           # check prediction shape


Running inference on validation data ...
Inference completed.
Predictions shape: (1684, 14)


In [ ]:
'''# -------------------------------------------
# 4.1 Inverse-transform targets to original units
#      Assumes scaler_targets was fitted on target_cols during preprocessing
# -------------------------------------------

# Y_val and Y_pred are currently in [-1, 1] scaled space
# Convert both back to original measurement units (cm, kg, etc.)
Y_true_orig = scaler_targets.inverse_transform(Y_val)
Y_pred_orig = scaler_targets.inverse_transform(Y_pred)'''

In [19]:

# -------------------------------------------
# 5. Compute metrics per body measurement
# -------------------------------------------
print("\nComputing metrics (R^2, MSE, MAE) for each body measurement ...")

results = []                                        # list to collect metric rows

# Loop over each target dimension / body measurement
for i, name in enumerate(target_cols):              # i = column index, name = column name
    y_true = Y_val[:, i]                            # true values for this measurement
    y_pred = Y_pred[:, i]                           # predicted values for this measurement

    r2  = r2_score(y_true, y_pred)                  # compute R^2 score
    mse = mean_squared_error(y_true, y_pred)        # compute mean squared error
    mae = mean_absolute_error(y_true, y_pred)       # compute mean absolute error

    # append metrics as a dict (one row)
    results.append({
        "body_measurement": name,                   # column for measurement name
        "r2": r2,                                   # R^2 value
        "mse": mse,                                 # MSE value
        "mae": mae                                  # MAE value
    })

    # print quick summary for this measurement
    print(f"{name:20s} -> R^2: {r2:.4f}, MSE: {mse:.6f}, MAE: {mae:.6f}")

# Convert results list to a DataFrame for nice tabular view
results_df = pd.DataFrame(results)                  # create DataFrame from list of dicts

print("\nPer-measurement metrics table:")
print(results_df)                                   # display table with all metrics



Computing metrics (R^2, MSE, MAE) for each body measurement ...
ankle                -> R^2: 0.7504, MSE: 0.249565, MAE: 0.389992
arm-length           -> R^2: 0.7341, MSE: 0.265920, MAE: 0.402745
bicep                -> R^2: 0.7821, MSE: 0.217850, MAE: 0.356737
calf                 -> R^2: 0.7657, MSE: 0.234275, MAE: 0.385369
chest                -> R^2: 0.8543, MSE: 0.145701, MAE: 0.291422
forearm              -> R^2: 0.8276, MSE: 0.172428, MAE: 0.323385
hip                  -> R^2: 0.8328, MSE: 0.167243, MAE: 0.312232
leg-length           -> R^2: 0.8213, MSE: 0.178735, MAE: 0.332222
shoulder-breadth     -> R^2: 0.8544, MSE: 0.145582, MAE: 0.307560
shoulder-to-crotch   -> R^2: 0.8464, MSE: 0.153613, MAE: 0.311521
thigh                -> R^2: 0.8116, MSE: 0.188374, MAE: 0.331441
waist                -> R^2: 0.8599, MSE: 0.140097, MAE: 0.285569
wrist                -> R^2: 0.6907, MSE: 0.309315, MAE: 0.440434
weight_kg            -> R^2: 0.8812, MSE: 0.118850, MAE: 0.265628

Per-measur

In [20]:
# -------------------------------------------
# 6. Compute overall (mean) scores across all measurements
# -------------------------------------------
overall_r2  = results_df["r2"].mean()               # mean R^2 over all body measurements
overall_mse = results_df["mse"].mean()              # mean MSE over all body measurements
overall_mae = results_df["mae"].mean()              # mean MAE over all body measurements

print("\nOverall (mean) scores across all body measurements:")
print(f"Mean R^2 : {overall_r2:.4f}")
print(f"Mean MSE : {overall_mse:.6f}")
print(f"Mean MAE : {overall_mae:.6f}")

# Optionally, add a final row with the overall mean scores to the table
overall_row = {
    "body_measurement": "OVERALL_MEAN",             # label row as overall
    "r2": overall_r2,
    "mse": overall_mse,
    "mae": overall_mae
}
results_df = pd.concat([results_df, pd.DataFrame([overall_row])], ignore_index=True)

print("\nMetrics table including overall mean row:")
print(results_df)                                   # final table with per-measurement + overall row



Overall (mean) scores across all body measurements:
Mean R^2 : 0.8080
Mean MSE : 0.191968
Mean MAE : 0.338304

Metrics table including overall mean row:
      body_measurement        r2       mse       mae
0                ankle  0.750435  0.249565  0.389992
1           arm-length  0.734080  0.265920  0.402745
2                bicep  0.782150  0.217850  0.356737
3                 calf  0.765725  0.234275  0.385369
4                chest  0.854299  0.145701  0.291422
5              forearm  0.827572  0.172428  0.323385
6                  hip  0.832757  0.167243  0.312232
7           leg-length  0.821265  0.178735  0.332222
8     shoulder-breadth  0.854418  0.145582  0.307560
9   shoulder-to-crotch  0.846387  0.153613  0.311521
10               thigh  0.811626  0.188374  0.331441
11               waist  0.859903  0.140097  0.285569
12               wrist  0.690685  0.309315  0.440434
13           weight_kg  0.881150  0.118850  0.265628
14        OVERALL_MEAN  0.808032  0.191968  0.33830

3.4. performance based on real values after inverse transforming

In [21]:
# -------------------------------------------
# 4.1 Inverse-transform targets to original units
# -------------------------------------------

Y_true_orig = scaler_targets.inverse_transform(Y_val)
Y_pred_orig = scaler_targets.inverse_transform(Y_pred)


In [22]:
# -------------------------------------------
# 5. Compute metrics per body measurement (in original units)
# -------------------------------------------
print("\nComputing metrics (R^2, MSE, MAE) for each body measurement in ORIGINAL UNITS ...")

results = []                                        # list to collect metric rows

# Loop over each target dimension / body measurement
for i, name in enumerate(target_cols):              # i = column index, name = column name
    # use inverse-transformed (real-unit) values
    y_true = Y_true_orig[:, i]                      # true values for this measurement (cm, kg, etc.)
    y_pred = Y_pred_orig[:, i]                      # predicted values for this measurement (cm, kg, etc.)

    r2  = r2_score(y_true, y_pred)                  # compute R^2 score
    mse = mean_squared_error(y_true, y_pred)        # compute mean squared error
    mae = mean_absolute_error(y_true, y_pred)       # compute mean absolute error

    # append metrics as a dict (one row)
    results.append({
        "body_measurement": name,                   # column for measurement name
        "r2": r2,                                   # R^2 value
        "mse": mse,                                 # MSE value (in squared real units)
        "mae": mae                                  # MAE value (in real units)
    })

    # print quick summary for this measurement
    print(f"{name:20s} -> R^2: {r2:.4f}, MSE: {mse:.6f}, MAE: {mae:.6f}")

# Convert results list to a DataFrame for nice tabular view
results_df = pd.DataFrame(results)                  # create DataFrame from list of dicts

print("\nPer-measurement metrics table (original units):")
print(results_df)                                   # display table with all metrics


Computing metrics (R^2, MSE, MAE) for each body measurement in ORIGINAL UNITS ...
ankle                -> R^2: 0.7504, MSE: 0.782260, MAE: 0.690461
arm-length           -> R^2: 0.7341, MSE: 2.432224, MAE: 1.218026
bicep                -> R^2: 0.7821, MSE: 2.968307, MAE: 1.316811
calf                 -> R^2: 0.7657, MSE: 2.098850, MAE: 1.153467
chest                -> R^2: 0.8543, MSE: 14.952309, MAE: 2.952189
forearm              -> R^2: 0.8276, MSE: 0.951584, MAE: 0.759697
hip                  -> R^2: 0.8328, MSE: 12.530473, MAE: 2.702634
leg-length           -> R^2: 0.8213, MSE: 4.076244, MAE: 1.586548
shoulder-breadth     -> R^2: 0.8544, MSE: 0.840340, MAE: 0.738930
shoulder-to-crotch   -> R^2: 0.8464, MSE: 2.889761, MAE: 1.351150
thigh                -> R^2: 0.8116, MSE: 5.451686, MAE: 1.783037
waist                -> R^2: 0.8599, MSE: 14.867647, MAE: 2.941826
wrist                -> R^2: 0.6907, MSE: 0.497475, MAE: 0.558556
weight_kg            -> R^2: 0.8812, MSE: 23.652269, MAE

In [23]:
# -------------------------------------------
# 6. Compute overall (mean) scores across all measurements (original units)
# -------------------------------------------
overall_r2  = results_df["r2"].mean()               # mean R^2 over all body measurements
overall_mse = results_df["mse"].mean()              # mean MSE over all body measurements
overall_mae = results_df["mae"].mean()              # mean MAE over all body measurements

print("\nOverall (mean) scores across all body measurements (original units):")
print(f"Mean R^2 : {overall_r2:.4f}")
print(f"Mean MSE : {overall_mse:.6f}")
print(f"Mean MAE : {overall_mae:.6f}")

# Optionally, add a final row with the overall mean scores to the table
overall_row = {
    "body_measurement": "OVERALL_MEAN",             # label row as overall
    "r2": overall_r2,
    "mse": overall_mse,
    "mae": overall_mae
}
results_df = pd.concat([results_df, pd.DataFrame([overall_row])], ignore_index=True)

print("\nMetrics table including overall mean row (original units):")
print(results_df)                                   # final table with per-measurement + overall row



Overall (mean) scores across all body measurements (original units):
Mean R^2 : 0.8080
Mean MSE : 6.356531
Mean MAE : 1.678612

Metrics table including overall mean row (original units):
      body_measurement        r2        mse       mae
0                ankle  0.750435   0.782260  0.690461
1           arm-length  0.734080   2.432224  1.218026
2                bicep  0.782150   2.968307  1.316811
3                 calf  0.765725   2.098850  1.153467
4                chest  0.854299  14.952309  2.952189
5              forearm  0.827572   0.951584  0.759697
6                  hip  0.832757  12.530473  2.702634
7           leg-length  0.821265   4.076244  1.586548
8     shoulder-breadth  0.854418   0.840340  0.738930
9   shoulder-to-crotch  0.846387   2.889761  1.351150
10               thigh  0.811626   5.451686  1.783037
11               waist  0.859903  14.867647  2.941826
12               wrist  0.690685   0.497475  0.558556
13           weight_kg  0.881150  23.652269  3.747231
14

4. Compare BMI

In [24]:
# indices of the columns that were StandardScaled (same order as fit)
std_cols = standard_feature_cols  # ['height_cm','volume']
std_idxs = [X.columns.get_loc(c) for c in std_cols]

# take BOTH scaled columns
X_std_scaled = X_val[:, std_idxs]              # shape (n, 2)

# inverse transform both
X_std_orig = scaler_standard_features.inverse_transform(X_std_scaled)

# pick height back out
height_pos = std_cols.index("height_cm")
height_cm_orig = X_std_orig[:, height_pos]
height_m_orig = height_cm_orig / 100.0


In [25]:
# -------------------------------------------
# Extract real weights (kg)
# -------------------------------------------

weight_idx = target_cols.index("weight_kg")

weight_true = Y_true_orig[:, weight_idx]
weight_pred = Y_pred_orig[:, weight_idx]


In [26]:
# -------------------------------------------
# BMI computation
# -------------------------------------------

bmi_true = weight_true / (height_m_orig ** 2)
bmi_pred = weight_pred / (height_m_orig ** 2)


In [27]:
# -------------------------------------------
# BMI MAE per record
# -------------------------------------------

bmi_mae_per_record = np.abs(bmi_true - bmi_pred)


In [28]:
average_bmi_mae = bmi_mae_per_record.mean()

print(f"Average BMI MAE across {len(bmi_mae_per_record)} records: {average_bmi_mae:.4f}")


Average BMI MAE across 1684 records: 1.2882


* MAE = 2.5 -> predictions are off (away) by about 2.5 units from the true values. Sometimes error can be above (overestimates) and sometimes error can be below (underestimated). On average model is wrong by 2.5 unots
* R Squared = 0.55 -> model explains about 55% of the variation in the overall body measurements
* MSE = 12.25 -> squared error between the predictions and true values are 12.25 units
* MSE < 0 ->  get smaller overall error by ignoring all inputs and just predicting the average hip size for everyone, instead of using your model’s predictions.
* MSE = -0.5 -> worse than the mean, with 50% more squared error than the mean